In [1]:
# %load ../../load_magic/storage.py

!mkdir ../data
!mkdir ../data/csv
!mkdir ../saves
!mkdir ../saves/pickle
!mkdir ../saves/csv
import pickle
import pandas as pd
import os

# Handy list of the different types of encodings
encoding = ['latin1', 'iso8859-1', 'utf-8'][1]

# Change this to your data and saves folders
data_folder = r'../data/'
saves_folder = r'../saves/'

def load_csv(csv_name=None):
    csv_folder = data_folder + 'csv/'
    if csv_name is None:
        
        # Get the newest csv
        csv_path = max([os.path.join(csv_folder, f) for f in os.listdir(csv_folder)],
                       key=os.path.getmtime)
        
    else:
        csv_path = csv_folder + csv_name + '.csv'
    csv_df = pd.read_csv(csv_path, encoding=encoding)
    
    return(csv_df)

def load_object(obj_name):
    pickle_path = saves_folder + 'pickle/' + obj_name + '.pickle'
    try:
        object = pd.read_pickle(pickle_path)
    except:
        with open(pickle_path, 'rb') as handle:
            object = pickle.load(handle)
    
    return(object)

def save_dataframes(include_index=False, **kwargs):
    csv_folder = saves_folder + 'csv/'
    for frame_name in kwargs:
        if isinstance(kwargs[frame_name], pd.DataFrame):
            csv_path = csv_folder + frame_name + '.csv'
            kwargs[frame_name].to_csv(csv_path, sep=',', encoding=encoding,
                                      index=include_index)

# Classes, functions, and methods cannot be pickled
def store_objects(**kwargs):
    for obj_name in kwargs:
        if hasattr(kwargs[obj_name], '__call__'):
            raise RuntimeError('Functions cannot be pickled.')
        obj_path = saves_folder + 'pickle/' + str(obj_name)
        pickle_path = obj_path + '.pickle'
        if isinstance(kwargs[obj_name], pd.DataFrame):
            kwargs[obj_name].to_pickle(pickle_path)
        else:
            with open(pickle_path, 'wb') as handle:
                pickle.dump(kwargs[obj_name], handle, pickle.HIGHEST_PROTOCOL)

mkdir: ../data: File exists
mkdir: ../data/csv: File exists
mkdir: ../saves: File exists
mkdir: ../saves/pickle: File exists
mkdir: ../saves/csv: File exists


In [2]:

jobs_df = load_csv()
jobs_df.columns
jobs_df.sample(4).T

Index(['Job Requisition', 'Management Level', 'Required Clearance',
       'Clearance Agency', 'Job Requisition Type', 'Group', 'Account Group',
       'IMT', 'Cluster', 'FSO', 'Resource Manager', 'Primary Recruiter',
       'Hiring Manager', 'Primary Location', 'Primary Location State/Province',
       'Primary Location Country', 'Job Description', 'Job Family'],
      dtype='object')

,2243,1671,1674,1619
Job Requisition,R0007339 Strategic Communications and Digital ...,R0014732 Laboratory Advisor (Open),R0014726 FMS Acquisition Program Cost Estimato...,"R0014969 Emergency Management Site Inspector, ..."
Management Level,Associate,Associate,Senior Consultant,Associate
Required Clearance,TS/SCI w/FSP,None,Secret,None
Clearance Agency,UGC,None,DOD,None
Job Requisition Type,Sold and Funded,Contingent,Sold and Funded,Contingent
Group,DEFENSE & INTELLIGENCE GROUP,CIVIL COMMERCIAL GROUP,DEFENSE & INTELLIGENCE GROUP,CIVIL COMMERCIAL GROUP
Account Group,NATL AGENCIES ACCT GROUP,HEALTH ACCT GROUP,INTL ACCT GROUP,JHT ACCT GROUP
IMT,NA CONSULTING IMT,HEALTH IMT,INTL FMS IMT,JHT IMT
Cluster,Wash Metro Cluster,Atlanta Cluster,Wash Metro Cluster,Norfolk Cluster
FSO,Consulting,Consulting,Consulting,Consulting


In [3]:

# Split the Job Requisition field into its ID
def f(x):
    state = str(x)
    comma_list = state.split(',')
    if len(comma_list) > 1:
        state = comma_list[1]
        state = state.strip()
    elif '8283 Greensboro Drive' in state:
        state = 'DC'
    else:
        state = pd.NaN
    
    return state

jobs_df['Job Requisition ID'] = jobs_df['Job Requisition'].map(lambda x: str(x).split(' ')[0])
jobs_df['Job Posting'] = jobs_df['Job Requisition'].map(lambda x: ' '.join(str(x).split(' ')[1:]))
#jobs_df['Primary Location State/Province'] = jobs_df['Location'].map(f)

In [4]:

# Get the Data Science job titles
full_list = sorted(jobs_df['Job Posting'].unique().tolist())
job_title_list = []
for job_title in full_list:
    if ('Senior' not in job_title) and ('.NET' not in job_title) and ('Chief' not in job_title) and ('Lead' not in job_title):
        if ('Data' in job_title) and ('Scien' in job_title):
            job_title_list.append(job_title)
        elif ('Machine' in job_title) and ('Learning' in job_title):
            job_title_list.append(job_title)
print(job_title_list)

['Accounting Data Scientist (Open)', 'Computer Vision and Machine Learning Developer, Junior (Open)', 'Data Analyst/Scientist (Open)', 'Data Analyst/Scientist, Mid (Open)', 'Data Science Intelligence Tradecraft Analyst (Open)', 'Data Scientist (Open)', 'Data Scientist, Junior (Open)', 'Data Scientist, Mid (Open)', 'Data Scientist/Consultant (Open)', 'Data Scientist/Intelligence Analyst (Open)', 'Deep Learning/Computer Vision Data Scientist (Open)', 'Graph Analytics Research Data Scientist (Open)', 'Intelligence Data Scientist (Open)', 'Machine Intelligence Data Scientist, Mid (Open)', 'Machine Learning/Data Scientist (Open)', 'Research Data Scientist (Open)']


In [19]:

# Get the security clearances
match_series = (jobs_df['Job Posting'].isin(job_title_list))
good_jobs_df = jobs_df[match_series].copy()
good_jobs_df['Required Clearance'].unique().tolist()

['None',
 'Secret',
 'Eligibility Determination Timeline',
 'TS/SCI',
 'Top Secret',
 'Confidential',
 'TS/SCI w/FSP',
 'TS/SCI w/CIP']

In [20]:

# Get rid of the full scope poly jobs
match_series = (good_jobs_df['Required Clearance'] == 'TS/SCI w/FSP')
good_jobs_df = good_jobs_df[~match_series]
good_jobs_df['Primary Location State/Province'].unique().tolist()

['Maryland',
 'District of Columbia',
 'Massachusetts',
 'Hawaii',
 'Virginia',
 'Alabama',
 'Illinois',
 'Texas',
 'Florida',
 'New York',
 'Baden-Wurttemberg',
 'California',
 'Nebraska']

In [21]:

# Get the jobs in states that we care about
location_list = ['District of Columbia', 'Maryland', 'Massachusetts', 'New York', 'Virginia', 'Colorado', 'Arizona']
#location_list = ['DC', 'MD', 'MA', 'VA']
match_series = (good_jobs_df['Primary Location State/Province'].isin(location_list))
column_list = ['Job Requisition ID', 'Job Posting', 'Management Level', 'Required Clearance', 'Clearance Agency',
               'Job Requisition Type', 'Group', 'Account Group', 'IMT', 'Cluster', 'FSO', 'Resource Manager',
               'Primary Recruiter', 'Hiring Manager', 'Primary Location', 'Primary Location State/Province',
               'Job Description', 'Job Family']
good_jobs_df = good_jobs_df[match_series][column_list]

In [23]:

# Get the jobs you haven't already applied for
already_applied_for_list = load_object('already_applied_for_list')
match_series = (good_jobs_df['Job Requisition ID'].isin(already_applied_for_list))
good_jobs_df = good_jobs_df[~match_series]
save_dataframes(good_jobs_df=good_jobs_df)

In [24]:

# Get the job description subheadings
import re

heading_regex = re.compile(r'([A-Z][A-za-z ]+:)')
description_list = sorted(good_jobs_df['Job Description'].unique().tolist())
set(heading_regex.findall(' '.join(description_list)))

{'Additional Qualification:',
 'Additional Qualifications:',
 'Basic Qualification:',
 'Basic Qualifications:',
 'Clearance:',
 'Experience with at least one of the following:',
 'Key Role:',
 'MA or MS degree Key Role:',
 'Preferred Qualifications:'}

In [25]:

# Compile all the subheading regexes
paragraph_regex = re.compile(r'\r *\r')
key_role_regex = re.compile(r'^(Key Role|The Challenge): *\r')
basic_qualifications_regex = re.compile(r'^(Basic Qualifications?|You Have): *\r')
additional_qualifications_regex = re.compile(r'^(Additional Qualifications?|Nice If You Have):? *\r')
clearance_regex = re.compile(r'^(Clearance): *\r')
preferred_qualifications_regex = re.compile(r'^(Preferred Qualifications): *\r')
relocation_regex = re.compile(r'^(Relocation): *\r')
perks_regex = re.compile(r'^(The Perks): *\r')
space_regex = re.compile(r' +')
r_regex = re.compile(r'\r')

In [26]:

# Cleaning routines
def clean(paragraph, regex):
    paragraph = regex.sub('', paragraph)
    paragraph = space_regex.sub(' ', paragraph)
    paragraph = r_regex.sub('\n', paragraph)
    paragraph = paragraph.strip()
    
    return paragraph

In [27]:

# Create subheading features from the job description
def f(row):
    description_list = paragraph_regex.split(row['Job Description'])
    last_attribute = 'No Idea'
    for paragraph in description_list:
        if key_role_regex.match(paragraph) is not None:
            last_attribute = 'Key Role'
            row[last_attribute] = clean(paragraph, key_role_regex)
        elif basic_qualifications_regex.match(paragraph) is not None:
            last_attribute = 'Basic Qualifications'
            row[last_attribute] = clean(paragraph, basic_qualifications_regex)
        elif additional_qualifications_regex.match(paragraph) is not None:
            last_attribute = 'Additional Qualifications'
            row[last_attribute] = clean(paragraph, additional_qualifications_regex)
        elif clearance_regex.match(paragraph) is not None:
            last_attribute = 'Clearance'
            row[last_attribute] = clean(paragraph, clearance_regex)
        elif preferred_qualifications_regex.match(paragraph) is not None:
            last_attribute = 'Preferred Qualifications'
            row[last_attribute] = clean(paragraph, preferred_qualifications_regex)
        elif relocation_regex.match(paragraph) is not None:
            last_attribute = 'Relocation'
            row[last_attribute] = clean(paragraph, relocation_regex)
        elif perks_regex.match(paragraph) is not None:
            last_attribute = 'Perks'
            row[last_attribute] = clean(paragraph, perks_regex)
        else:
            row[last_attribute] = row[last_attribute] + '\n\n' + paragraph
    
    return row

In [29]:

# Get the requiremnents list from the basic qualifications
dash_regex = re.compile(r'[\n_]​?-')
good_jobs_df = good_jobs_df.apply(f, axis=1)
bq_series = good_jobs_df['Basic Qualifications'].map(lambda x: dash_regex.split('\n' + str(x))[1:])
max_reqs = bq_series.map(lambda x: len(x)).max()
req_list = []
for bq_list in bq_series.tolist():
    for requirement in bq_list:
        requirement = requirement.strip()
        if requirement not in req_list:
            req_list.append(requirement)

In [30]:

# Create the basic qualifications dataframe
column_list = ['requisition_number'] + ['qualification_'+str(x).zfill(2) for x in range(1, max_reqs+1)]
rows_list = []

def f(row):
    row_dict = {}
    row_dict['requisition_number'] = row['Job Requisition ID']
    quals_list = [x.strip() for x in dash_regex.split('\n' + str(row['Basic Qualifications']))[1:]]
    for x, qual in enumerate(quals_list):
        row_dict['qualification_'+str(x).zfill(2)] = qual
    rows_list.append(row_dict)
    
_ = good_jobs_df.apply(f, axis=1)
basic_qualifications_df = pd.DataFrame(rows_list, columns=column_list)

In [31]:

# Create the qual ratings widget dataset
column_list = ['req_number', 'qual_description']
rows_list = []

def f(row):
    for qual in dash_regex.split('\n' + str(row['Basic Qualifications']))[1:]:
        row_dict = {}
        row_dict['req_number'] = row['Job Requisition ID']
        row_dict['qual_description'] = qual.strip()
        rows_list.append(row_dict)
    
_ = good_jobs_df.apply(f, axis=1)
bqs_df = pd.DataFrame(rows_list, columns=column_list)
bqs_df['qual_rating'] = -1
qual_description_dict = load_object('qual_description_dict')

def f(row):
    if row['qual_description'] in qual_description_dict.keys():
        row['qual_rating'] = qual_description_dict[row['qual_description']]
    
    return row

bqs_df = bqs_df.apply(f, axis=1)

In [32]:

# Define the qual rating business logic for the interactive widget app
def get_req(**kwargs):
    if kwargs['rating_buttons'] is not None:
        qual_rating = rating_options.index(kwargs['rating_buttons'])
        index = int(kwargs['index'])
        qual_description = kwargs['rating_text']
        match_series = (bqs_df['qual_description'] == qual_description)
        bqs_df.loc[match_series, 'qual_rating'] = qual_rating
        qual_description_dict[qual_description] = qual_rating
        store_objects(qual_description_dict=qual_description_dict)
        
        match_series = (bqs_df['qual_rating'] == -1)
        display(bqs_df[match_series].shape)
        df = bqs_df[match_series]
        if df.shape[0] > 0:
            row_series = df.sample(1)
            req_number = row_series['req_number'].squeeze()
            qual_description = row_series['qual_description'].squeeze()
            hidden_label.value = str(row_series.index.max())
            rating_text.value = qual_description
            rating_text.description = req_number+':'
            rating_buttons.value = None
        else:
            hidden_label.value = str(-1)
            rating_text.value = 'Finished'
            rating_text.description = 'XXXXXXXXX:'
            rating_buttons.value = None
    display(kwargs)

In [33]:

# Prep the widget app for first use
match_series = (bqs_df['qual_rating'] == -1)
df = bqs_df[match_series]
if df.shape[0] > 0:
    row_series = df.sample(1)
    req_number = row_series['req_number'].squeeze()
    qual_description = row_series['qual_description'].squeeze()
    row_index = row_series.index.max()
else:
    req_number = 'XXXXXXXXX'
    qual_description = 'Finished'
    row_index = -1

In [34]:

# Prep the widgets themselves for first use
import ipywidgets
from ipywidgets import HTML, RadioButtons, Label, Layout, interactive
from IPython.display import display

kwargs = {}
kwargs['index'] = Label(value=str(row_index), layout=Layout(visibility='hidden'))
kwargs['rating_text'] = HTML(
    value=qual_description,
    description=req_number+':'
    )
rating_options = ['Not at all', 'Very little', 'Neither', 'Very much', 'Completely']
kwargs['rating_buttons'] = RadioButtons(
    options=rating_options,
    description='Applicability:',
    disabled=False,
    button_style='',
    tooltips=rating_options,
    value=None
    )
ip = interactive(get_req, {'manual': True}, **kwargs)
for i, widget in enumerate(ip.children):
    if type(widget) == ipywidgets.widgets.widget_selection.RadioButtons:
        rating_buttons = ip.children[i]
    elif type(widget) == ipywidgets.widgets.widget_string.HTML:
        rating_text = ip.children[i]
    elif type(widget) == ipywidgets.widgets.widget_string.Label:
        hidden_label = ip.children[i]
    elif type(widget) == ipywidgets.widgets.widget_button.Button:
        submit_button = ip.children[i]
        submit_button.description = 'Submit'
    elif type(widget) == ipywidgets.widgets.widget_output.Output:
        out = ip.children[i]

In [35]:

# Display the app
from ipywidgets import VBox, HBox

display(VBox([rating_text, HBox([rating_buttons, submit_button]), out]))

VBox(children=(HTML(value='Experience with machine learning, data mining, statistics, or graph algorithms in an academic environment or internship', description='R0017048:'), HBox(children=(RadioButtons(description='Applicability:', options=('Not at all', 'Very little', 'Neither', 'Very much', 'Completely'), value=None), Button(description='Submit', style=ButtonStyle()))), Output()))

In [36]:

# Display jobs you are qualified for
bqs_group = bqs_df.groupby('req_number')
bqs_group_means_df = bqs_group.mean()
match_series = (bqs_group_means_df['qual_rating'] >= 3.75)
req_number_list = bqs_group_means_df[match_series].sort_values('qual_rating', ascending=False).index.tolist()
match_series = good_jobs_df['Job Requisition ID'].isin(req_number_list)


# Get rid of useless columns
column_list = ['Job Requisition ID', 'Job Posting', 'Key Role', 'Basic Qualifications', 'Additional Qualifications',
               'Clearance', 'Required Clearance', 'Primary Location', 'Primary Location State/Province', 'Job Family']
best_jobs_df = good_jobs_df[match_series][column_list].T.dropna()
save_dataframes(include_index=True, best_jobs_df=best_jobs_df)
best_jobs_df

,215,1065,1066,2329
Job Requisition ID,R0018879,R0017065,R0017064,R0004444
Job Posting,Computer Vision and Machine Learning Developer...,Data Analyst/Scientist (Open),Data Analyst/Scientist (Open),Data Scientist (Open)
Key Role,"Perform engineering work, including planning, ...",Leverage technical and analytic expertise to e...,Apply technical and analytic expertise to expl...,Analyze large data sets and develop automated ...
Basic Qualifications,-Experience with software development language...,"-Experience with machine learning, data mining...","-Experience with machine learning, data mining...",-Experience in the data science and analytics ...
Additional Qualifications,-Experience with other programming languages a...,"-Experience with querying databases, programma...",-Experience with querying databases programmat...,-Experience with deep learning\n-Experience wi...
Clearance,Applicants selected will be subject to a secur...,Applicants selected will be subject to a secur...,Applicants selected will be subject to a secur...,Applicants selected will be subject to a secur...
Required Clearance,Secret,Eligibility Determination Timeline,Eligibility Determination Timeline,Secret
Primary Location,"USA, DC, Washington (901 15th St NW)","USA, VA, Alexandria (6361 Walker Ln)","USA, VA, Alexandria (6361 Walker Ln)","USA, VA, Alexandria (6354 Walker Ln)"
Primary Location State/Province,District of Columbia,Virginia,Virginia,Virginia
Job Family,Electrical and Electronics Engineer,SD Process Engineer,SD Process Engineer,Data Scientist - Mathematics Specialization


In [42]:

%pprint

# Add req numbers (after the process) to your already-applied list
already_applied_for_list = list(set(['R0004444', 'R0013123', 'R0011910', 'R0006735', 'R0018879', 'R0017065', 'R0017064', 'R0004444']) | set(already_applied_for_list))
store_objects(already_applied_for_list=already_applied_for_list)
already_applied_for_list

Pretty printing has been turned OFF


['R0018879', 'R0004444', 'R0017065', 'R0013123', 'R0011910', 'R0006735', 'R0017064']


# Admin Stuff

In [ ]:

best_jobs_df.shape

In [ ]:

column_list = ['Job Requisition ID', 'Job Posting', 'Key Role', 'Basic Qualifications', 'Additional Qualifications',
               'Clearance', 'Required Clearance', 'Primary Location State/Province', 'Job Family']
#good_jobs_df = load_csv('good_jobs_df')
good_jobs_df.columns

In [ ]:

match_series = (good_jobs_df['Job Requisition ID'] == 'R0012921')
good_jobs_df[match_series].T

In [ ]:

match_series = (bqs_df['req_number'] == 'R0012921')
bqs_df[match_series]['qual_description'].tolist()

In [ ]:

column_list = ['req_number', 'qual_description']
rows_list = []

def f(row):
    for qual in dash_regex.split('\n' + str(row['Additional Qualifications']))[1:]:
        row_dict = {}
        row_dict['req_number'] = row['Job Requisition ID']
        row_dict['qual_description'] = qual.strip()
        rows_list.append(row_dict)
    
_ = good_jobs_df.apply(f, axis=1)
aqs_df = pd.DataFrame(rows_list, columns=column_list)
match_series = (aqs_df['req_number'] == 'R0012921')
aqs_df[match_series]['qual_description'].tolist()

In [ ]:

#best_jobs_df = load_csv('best_jobs_df')
best_req_list = best_jobs_df.columns.tolist()[1:]
match_series = (jobs_df['Job Requisition ID'].isin(best_req_list))
column_list = ['Job Requisition ID', 'Job Posting', 'Resource Manager', 'Primary Recruiter', 'Hiring Manager', 
               'Primary Location State/Province']
lack_of_work_df = jobs_df[match_series][column_list].sort_values('Resource Manager')
save_dataframes(lack_of_work_df=lack_of_work_df)

In [ ]:

lack_of_work_df